In [94]:
import pandas as pd 
import numpy as np 
# Kristel Tan (ktan@bu.edu) & Nisa Gurung (ngurung@bu.edu)
# CS505-final-project : data-collection.py
# This file contains scripts for intial results of data collection.

movie_df = pd.read_csv('movie_metadata.csv')

# Deleting columns that are not useful 
delete = ['color', 'num_critic_for_reviews', 'actor_3_facebook_likes', 'actor_1_facebook_likes','movie_imdb_link', 'country', 'language', 
         'aspect_ratio', 'num_user_for_reviews', 'num_voted_users', 'actor_2_facebook_likes', 'content_rating']

for d in delete: 
    del movie_df[d]

# Delete rows with empty cells for any column     
for c in movie_df: 
    movie_df[c].replace('', np.nan, inplace=True)
    movie_df.dropna(subset=[c], inplace=True)

movie_df = movie_df.reset_index(drop=True)

genres_set = set() 
for i in range(len(movie_df)): 
    g = movie_df['genres'][i].split('|')
    genres_set.update(g)

genres = list(genres_set)
genres = dict.fromkeys(genres)
genres_ct = {key: 0 for key in genres} 

for i in range(len(movie_df)): 
    g = movie_df['genres'][i].split('|')
    for item in g: 
        if item in genres: 
            genres_ct[item] += 1 

#Get the top five genres 
sortedCounts = [(genres_ct[key], key) for key in genres_ct]
sortedCounts.sort()
sortedCounts.reverse() 
top_genres = [item[1] for item in sortedCounts[:5]] 

# find the list of index of the rows that need to dropped 
topg_set = set(top_genres) 
drop = [] 
for i in range(len(movie_df)):
    g = set(movie_df['genres'][i].split('|'))
    if len(set.intersection(g, topg_set)) == 0:
        drop.append(i)

movie_df = movie_df.drop(movie_df.index[drop])
movie_df = movie_df.reset_index(drop=True) 
movie_df.to_csv('movieClean_metadata.csv')



done
